# How to build a simple Agent LLM App with LangGraph
* Very basic tool-using Agent LLM App with memory.

Language models can't do anything on their own; they can only create text based on what you ask them. However, LangChain allows people to build agents—think of these as smart systems or helpers— that use language models to think and decide what to do next.

Here’s how it works:
1. **Use the Language Model as a Brain**: The agent uses the language model to figure out which actions it should take, based on the information it has or what you ask it to do.
2. **Taking Action**: After deciding, the agent then goes ahead and does those actions.
3. **Learning and Adjusting**: Once the actions are done, the results can be given back to the language model. This helps the model check if everything is complete or if it needs to do something else.

So, essentially, LangChain helps turn a language model from just a tool for writing and answering into a system that can act and react, almost like a very simple robot brain.

Here we will build an agent that can interact with a search engine. You will be able to ask this agent questions, watch it calling the search tool, and have conversations with it.

## Concepts included
* [LangGraph](https://python.langchain.com/v0.2/docs/concepts/#langgraph).

LangGraph is a library created by the LangChain team for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows.

## Setup

#### After you download the code from the github repository in your computer
In terminal:
* cd project_name
* pyenv local 3.11.4
* poetry install
* poetry shell

#### To open the notebook with Jupyter Notebooks
In terminal:
* jupyter lab

Go to the folder of notebooks and open the right notebook.

#### To see the code in Virtual Studio Code or your editor of choice.
* open Virtual Studio Code or your editor of choice.
* open the project-folder
* open the 001-simple-agent.py file

## Create your .env file
* In the github repo we have included a file named .env.example
* Rename that file to .env file and here is where you will add your confidential api keys. Remember to include:
* OPENAI_API_KEY=your_openai_api_key
* LANGCHAIN_TRACING_V2=true
* LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
* LANGCHAIN_API_KEY=your_langchain_api_key
* LANGCHAIN_PROJECT=your_project_name

We will call our LangSmith project **001-simple-agent**.

## Connect with the .env file located in the same directory of this notebook

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [1]:
#!pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=google_api_key)

KeyError: 'OPENAI_API_KEY'

#### Install LangChain

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain

## Connect with an LLM and start a conversation with it

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#!pip install langchain-openai

* For this project, we will use OpenAI's gpt-3.5-turbo

#### Track the operation in LangSmith
* [Open LangSmith here](smith.langchain.com)

## Agents
* Agents use LLMs as reasoning engines to determine which actions to take.

## Tool-using Agent
* For this basic agent we will use just one tool. In next advanced projects, you will learn how to use agents with several tools.
* **Our tool of choice will be Tavily** - a search engine. 
#### Tavily API Key
* You will need to add your Tavily API key in the .env file. Obtain a key by signing up on their [website](https://tavily.com/).

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#pip install langgraph

#### Define tools

If you are using the pre-loaded poetry shell, you do not need to install the following package because it is already pre-loaded for you:

In [ ]:
#pip install langchain-community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults(max_results=2)
search.invoke("Who are the top stars of the 2024 Eurocup?")

[{'url': 'https://www.planetfootball.com/lists-and-rankings/euro-2024-player-of-the-tournament-power-ranking-kroos-bellingham',
  'content': 'Euro 2024 Player of the Tournament Power Ranking: Spain stars dominate ahead of England final. England will play Spain in the final of Euro 2024 and the Player of the Tournament award remains up for grabs. Gianluigi Donnarumma won the Player of the Tournament award last time out, following in the footsteps of Antoine Griezmann, Andres Iniesta ...'},
 {'url': 'https://www.theringer.com/soccer/2024/6/12/24176241/euro-2024-power-rankings',
  'content': "The Ringer's Euro 2024 Power Rankings Our guide to all 24 teams, key players to watch, and our favorite bets"}]

In [ ]:
tools = [search]

* **In order to enable this model to do tool calling we use .bind_tools** to give the language model knowledge of these tools:

In [ ]:
llm_with_tools = llm.bind_tools(tools)

## Create the agent
* We will be using LangGraph to construct the agent. 
* **Note that below we are passing in the origina chat model, not the llm_with_tools we built later**. That is because create_tool_calling_executor will call .bind_tools for us under the hood.

In [ ]:
# from langgraph.prebuilt import chat_agent_executor

# agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools)

* In the latest version of langgraph, we have a most modern way to do this:

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

## Run the agent
* Let's first try it with .invoke():

In [ ]:
from langchain_core.messages import HumanMessage

response = agent_executor.invoke({"messages": [HumanMessage(content="Where is the soccer Eurocup 2024 played?")]})

response["messages"]

[HumanMessage(content='Where is the soccer Eurocup 2024 played?', id='e221c286-c648-427d-875f-55a4be4b6dd7'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_e1kmnOzyjyK3HIf3SFVoXyc5', 'function': {'arguments': '{"query":"soccer Eurocup 2024 location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 92, 'total_tokens': 117}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9a556c37-d1f8-44d2-8dec-503a87bb04b0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'soccer Eurocup 2024 location'}, 'id': 'call_e1kmnOzyjyK3HIf3SFVoXyc5', 'type': 'tool_call'}], usage_metadata={'input_tokens': 92, 'output_tokens': 25, 'total_tokens': 117}),
 ToolMessage(content='[{"url": "https://apnews.com/article/euro-2024-germany-stadiums-9b6953fa87d168a754df5b2480b75010", "content": "File - The Cologne

* Now let's try it with .stream():

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="When and where will it be the 2024 Eurocup final match?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_kRA0CVP8kvkHZIVDlqozOhqy', 'function': {'arguments': '{"query":"2024 Eurocup final match date and location"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 96, 'total_tokens': 122}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ba5d36b9-fbb8-4c6d-98cf-c79c390147b9-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 Eurocup final match date and location'}, 'id': 'call_kRA0CVP8kvkHZIVDlqozOhqy', 'type': 'tool_call'}], usage_metadata={'input_tokens': 96, 'output_tokens': 26, 'total_tokens': 122})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://en.wikipedia.org/wiki/UEFA_Euro_2024_Final", "content": "The UEFA Euro 2024 final was a football match that determined the winners of UEFA E

## Adding memory
* Adding memory in LangGraph is very similar to what we did with LangChain.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

* Let's create our new agent with memory and set one thread_id so the agent can create a memory for each session as we did with our previous conversational RAG app:

In [ ]:
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "001"}}

* Let's now try this new agent with .stream():

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who won the 2024 soccer Eurocup?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DsiICrQERrNQt3RTtNn4Y6Tk', 'function': {'arguments': '{"query":"2024 soccer Eurocup winner"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 91, 'total_tokens': 114}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-66f5dc0b-40b7-4f0e-9d9f-3866964991e2-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': '2024 soccer Eurocup winner'}, 'id': 'call_DsiICrQERrNQt3RTtNn4Y6Tk', 'type': 'tool_call'}], usage_metadata={'input_tokens': 91, 'output_tokens': 23, 'total_tokens': 114})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.sportingnews.com/us/soccer/news/who-won-euro-2024-spain-england-oyarzabal/19ab0b3bf663194bc5193e50", "content": "Spain won the 2024 European Championship, becoming the 

In [ ]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Who were the top stars of that winner team?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_u8PyPAVf0Ufdw4JXvmy7OeDN', 'function': {'arguments': '{"query":"Spain Euro 2024 winning team top players"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 394, 'total_tokens': 420}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-ffee6e72-dcd0-4854-8c5a-147921b15687-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'Spain Euro 2024 winning team top players'}, 'id': 'call_u8PyPAVf0Ufdw4JXvmy7OeDN', 'type': 'tool_call'}], usage_metadata={'input_tokens': 394, 'output_tokens': 26, 'total_tokens': 420})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.cnn.com/2024/07/14/sport/spain-england-euro-2024-final-spt-intl/index.html", "content": "Spain players celebrate after defeating England a

* Let's change the thread_id and see what happens:

In [ ]:
config = {"configurable": {"thread_id": "002"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="About what soccer team we were talking?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='You mentioned the soccer team "Manchester United" in the previous conversation. How can I assist you with information about Manchester United?', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 89, 'total_tokens': 115}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3ffa2c4b-df91-45d0-873d-7a86f57f26ad-0', usage_metadata={'input_tokens': 89, 'output_tokens': 26, 'total_tokens': 115})]}}
----


* As you can see, when we changed the threat_id we started a different conversation memory, so our app does not remember the previous interaction and instead of giving us the right answer it decided to hallucinate.

## How to execute the code from Visual Studio Code
* In Visual Studio Code, see the file 001-simple-agent.py
* In terminal, make sure you are in the directory of the file and run:
    * python 001-simple-agent.py